In [21]:
import pandas as pd
janjune15=pd.read_csv("/Users/fezzibasma/Desktop/Data Full-Stack/Machine Learning Unsupervised/Projet Uber/uber-trip-data/uber-raw-data-janjune-15.csv")
janjune15.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
0,B02617,2015-05-17 09:47:00,B02617,141
1,B02617,2015-05-17 09:47:00,B02617,65
2,B02617,2015-05-17 09:47:00,B02617,100
3,B02617,2015-05-17 09:47:00,B02774,80
4,B02617,2015-05-17 09:47:00,B02617,90


In [22]:
split_datetime=pd.DataFrame(janjune15['Pickup_date'].str.split(' ').tolist(),
                         columns = ['Date','Time'])

In [23]:
split_datetime

,Date,Time
0,2015-05-17,09:47:00
1,2015-05-17,09:47:00
2,2015-05-17,09:47:00
3,2015-05-17,09:47:00
4,2015-05-17,09:47:00
...,...,...
14270474,2015-05-08,15:43:00
14270475,2015-05-08,15:43:00
14270476,2015-05-08,15:43:00
14270477,2015-05-08,15:44:00


In [24]:
#Formatter la date dans le format JJ/MM/AAAA au lieu de AAAA/MM/JJ 
from datetime import datetime
d=pd.to_datetime(split_datetime['Date'],format='%Y-%m-%d')
split_datetime['Date']=d.dt.strftime("%d/%m/%y")
split_datetime.head()

,Date,Time
0,17/05/15,09:47:00
1,17/05/15,09:47:00
2,17/05/15,09:47:00
3,17/05/15,09:47:00
4,17/05/15,09:47:00


In [25]:
data15=pd.concat([janjune15,split_datetime],axis=1)
data15.shape

(14270479, 6)

## Converger les resultats avec Taxi Zone pour obtenir le détail de l'historique data janjune15

In [26]:
taxi_zone_coordinates=pd.read_csv("/Users/fezzibasma/Desktop/Data Full-Stack/Machine Learning Unsupervised/Projet Uber/export_step/formatting_taxizones.csv",index_col=[0])
taxi_zone_coordinates=taxi_zone_coordinates.rename(columns={"LocationID": "locationID","zone":"Zone","borough":"Borough"})
taxi_zone_coordinates

,Zone,locationID,Borough,Lon,Lat
0,Newark Airport,1,EWR,-74.18,40.69
1,Jamaica Bay,2,Queens,-73.82,40.64
2,Allerton/Pelham Gardens,3,Bronx,-73.85,40.87
3,Alphabet City,4,Manhattan,-73.97,40.73
4,Arden Heights,5,Staten Island,-74.17,40.56
...,...,...,...,...,...
258,Williamsburg (South Side),256,Brooklyn,-73.96,40.71
259,Woodlawn/Wakefield,259,Bronx,-73.85,40.91
260,Woodside,260,Queens,-73.90,40.76
261,World Trade Center,261,Manhattan,-74.01,40.70


In [27]:
result15=data15.merge(taxi_zone_coordinates, on='locationID', how='left')
result15.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,Date,Time,Zone,Borough,Lon,Lat
0,B02617,2015-05-17 09:47:00,B02617,141,17/05/15,09:47:00,Lenox Hill West,Manhattan,-73.96,40.76
1,B02617,2015-05-17 09:47:00,B02617,65,17/05/15,09:47:00,Downtown Brooklyn/MetroTech,Brooklyn,-73.99,40.70
2,B02617,2015-05-17 09:47:00,B02617,100,17/05/15,09:47:00,Garment District,Manhattan,-73.99,40.75
3,B02617,2015-05-17 09:47:00,B02774,80,17/05/15,09:47:00,East Williamsburg,Brooklyn,-73.93,40.73
4,B02617,2015-05-17 09:47:00,B02617,90,17/05/15,09:47:00,Flatiron,Manhattan,-74.00,40.74


In [28]:
result15=result15.rename(columns={"Affiliated_base_num": "Base"}).dropna()

In [29]:
result15.head()

,Dispatching_base_num,Pickup_date,Base,locationID,Date,Time,Zone,Borough,Lon,Lat
0,B02617,2015-05-17 09:47:00,B02617,141,17/05/15,09:47:00,Lenox Hill West,Manhattan,-73.96,40.76
1,B02617,2015-05-17 09:47:00,B02617,65,17/05/15,09:47:00,Downtown Brooklyn/MetroTech,Brooklyn,-73.99,40.70
2,B02617,2015-05-17 09:47:00,B02617,100,17/05/15,09:47:00,Garment District,Manhattan,-73.99,40.75
3,B02617,2015-05-17 09:47:00,B02774,80,17/05/15,09:47:00,East Williamsburg,Brooklyn,-73.93,40.73
4,B02617,2015-05-17 09:47:00,B02617,90,17/05/15,09:47:00,Flatiron,Manhattan,-74.00,40.74


In [30]:
mycolumns=['Date','Time','locationID','Base','Lon','Lat','Borough','Zone']
result15=result15[mycolumns]
result15.head()

,Date,Time,locationID,Base,Lon,Lat,Borough,Zone
0,17/05/15,09:47:00,141,B02617,-73.96,40.76,Manhattan,Lenox Hill West
1,17/05/15,09:47:00,65,B02617,-73.99,40.70,Brooklyn,Downtown Brooklyn/MetroTech
2,17/05/15,09:47:00,100,B02617,-73.99,40.75,Manhattan,Garment District
3,17/05/15,09:47:00,80,B02774,-73.93,40.73,Brooklyn,East Williamsburg
4,17/05/15,09:47:00,90,B02617,-74.00,40.74,Manhattan,Flatiron


In [31]:
file_name = '/Users/fezzibasma/Desktop/Data Full-Stack/Machine Learning Unsupervised/Projet Uber/export_step/formatting_janjune15.csv'
result15.to_csv(file_name)
print('File is written to CSV File successfully.')

File is written to CSV File successfully.
